# Praca domowa nr 5

Budowanie zbioru modeli Rashomon.

Modele są zbudowany na podstawie preprocessingu przedstawionego w artykule: https://academic.oup.com/jamiaopen/article/1/1/87/5032901. 

Kod do artykułu dostępny jest pod linkiem: https://github.com/illidanlab/urgent-care-comparative

Zadanie: problem klasyfikacji, predykcja śmiertelności na podstawie przedstawienia danych w postaci *X48* (wg. artykułu powyżej).

### Biblioteki, wczytanie danych, dodatkowe funkcje

In [1]:
import numpy as np
import pandas as pd

import pickle
import os.path

import xgboost as xgb

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import auc as auc_score
from sklearn.utils import shuffle

In [2]:
# funkcja z repozytorium reprodukowanego artykułu
def balanced_subsample(x,y,subsample_size=1.0):
    class_xs = []
    min_elems = None
    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]
    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)
    xs = []
    ys = []
    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            np.random.shuffle(this_xs)

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = np.concatenate(xs)
    ys = np.concatenate(ys)

    return xs,ys

### Załadowanie danych po preprocessingu

In [3]:
X = np.load("X48.npy")

In [4]:
with open('y', 'rb') as f:
    labels = pickle.load(f)
    
task = [yy[0] for yy in labels]
y = np.array(task)

### Generowanie próbek do kroswalidacji

Przy modelowaniu skorzystamy z pięciokrotnej kroswalidacji - w celu zapewnienia reprodukowalności, indeksy użytych próbek można wczytać z pliku:

In [5]:
def get_cv_samples_indexes(X, y):
    if os.path.isfile('samples.npy'):
        return np.load("samples.npy", allow_pickle = True)
    else:
        tab = []
        skf = StratifiedKFold(n_splits = 5)
        
        for train_index, test_index in skf.split(X, y):
            tab.append((train_index, test_index))
            
        with open('samples.npy', 'wb') as f:
            pickle.dump(tab, f)
            
        return tab

In [6]:
cv_tab = get_cv_samples_indexes(X, y)

### Obiekty - model, random search, siatka hiperparametrów

In [7]:
model = xgb.XGBClassifier(objective='binary:logistic', n_jobs = -1, eval_metric = 'auc', use_label_encoder = False, seed = 123)

Zakres hiperparametrów wzorowany artykułem (tabela 1): https://jmlr.org/papers/volume20/18-444/18-444.pdf

Dokumentacja parametrów: https://xgboost.readthedocs.io/en/latest/parameter.html

In [8]:
hyperparameters =  {
    'learning_rate' : np.linspace(0.001, 1, num = 1000),
    'subsample' : np.linspace(0.1, 1, num = 100),
    'booster' : ['gbtree', 'dart'],
    'max_depth' : list(range(1, 20 + 1)),
    'min_child_weight' : np.linspace(0.001, 10, num = 100),
    'colsample_bytree' : np.linspace(0.001, 1, num = 100),
    'colsample_bylevel' : np.linspace(0.001, 1, num = 100),
    'lambda' : np.linspace(1, 10, num = 100),
    'alpha' : np.linspace(1, 10, num = 100),
    'n_estimators' : list(range(30, 700, 10))
}

In [9]:
# cross-validation generator in order to use balanced_subsample, as in the article

class cross_val_gen:
    def __init__(self, cv_tab):
        self.n_splits = 5
        self.cv_tab = cv_tab

    def split(self, X, y, groups=None):
        for train_index, test_index in cv_tab:
            yield train_index, test_index 

    def get_n_splits(self, X, y, groups=None):
        return self.n_splits

In [10]:
cv_search_obj = RandomizedSearchCV(estimator = model, param_distributions = hyperparameters, 
                            n_iter = 1, scoring = 'roc_auc', cv = cross_val_gen(cv_tab), return_train_score = True)

### Modelowanie

In [11]:
search = cv_search_obj.fit(X, y)

### Ramka danych wynikowych

In [12]:
results = pd.DataFrame(search.cv_results_)

In [13]:
results.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_subsample', 'param_n_estimators', 'param_min_child_weight',
       'param_max_depth', 'param_learning_rate', 'param_lambda',
       'param_colsample_bytree', 'param_colsample_bylevel', 'param_booster',
       'param_alpha', 'params', 'split0_test_score', 'split1_test_score',
       'split2_test_score', 'split3_test_score', 'split4_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score',
       'split0_train_score', 'split1_train_score', 'split2_train_score',
       'split3_train_score', 'split4_train_score', 'mean_train_score',
       'std_train_score'],
      dtype='object')

In [14]:
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_min_child_weight,param_max_depth,param_learning_rate,param_lambda,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,4.712036,0.491972,0.011764,0.001299,0.581818,630,4.344,2,0.404,8.45455,...,0.850605,0.009261,1,0.945541,0.944372,0.947325,0.945123,0.947248,0.945922,0.001176


In [15]:
with open('results.npy', 'wb') as f:
    pickle.dump(results, f)

In [16]:
results.to_csv("results.csv")